In [3]:
# xcube_sh imports
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube_sh.observers import Observers
from xcube_sh.viewer import ViewerServer

# xcube imports
from xcube.core.maskset import MaskSet
from xcube.core.geom import mask_dataset_by_geometry
from xcube.core.geom import clip_dataset_by_geometry

# Various utilities
import json
import numpy as np
import xarray as xr
import shapely.geometry
import IPython.display
import zarr
import os
import imageio
import geopandas
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'xcube_sh'

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#bbox2 cancun
#x1 = -86.83  # degree
#y1 = 21.00  # degree
#x2 = -86.74  # degree
#y2 = 21.16  # degree

#bbox1 macro-cancun
#x1 = -86.83  # degree
#y1 = 21.00  # degree
#x2 = -86.74  # degree
#y2 = 21.16  # degree


#bbox3 isla del carmen
#x1 = -91.96  # degree
#y1 = 18.50  # degree
#x2 = -91.56  # degree
#y2 = 18.80  # degree


#bbox = x1, y1, x2, y2
bbox1 = (-86.83, 21.00, -86.74, 21.16)
bbox2 = (-86.89, 21.03, -86.74, 21.45)
bbox3 = (-91.96, 18.50, -91.56, 18.80)

In [ ]:
IPython.display.GeoJSON(shapely.geometry.box(*bbox2).__geo_interface__)

In [ ]:
spatial_res = 0.00018   # = 20.038 meters in degree>

In [ ]:
#cube 1 2019
cube_config = CubeConfig(dataset_name='S2L2A',
                         band_names=['B04', 'B05', 'B06', 'B11', 'SCL', 'CLD'],
                         tile_size=[512, 512],
                         geometry=bbox2,
                         spatial_res=spatial_res,
                         time_range=['2020-02-01', '2020-11-30'],
                         time_period='5D')

In [ ]:
request_collector = Observers.request_collector()

In [ ]:
cube = open_cube(cube_config, observer=request_collector)

In [ ]:
#masking land

import xcube.core.maskset as maskset

scene_classif = maskset.MaskSet(cube.SCL)

In [ ]:
water_cube = cube.where(scene_classif.water)

In [ ]:
#chl calculation

b_from = water_cube.B04
b_peek = water_cube.B05
b_to = water_cube.B06

wlen_from = b_from.attrs['wavelength']
wlen_peek = b_peek.attrs['wavelength']
wlen_to = b_to.attrs['wavelength']

f = (wlen_peek - wlen_from) / (wlen_to - wlen_from)
mci = (b_peek - b_from) - f * (b_to - b_from)

############################################################


mci.attrs['long_name'] = 'Maximum Chlorophyll Index'
mci.attrs['units'] = 'unitless'


In [ ]:
cancun_cube = xr.Dataset({'mci': mci})

In [ ]:
#mci.time

In [ ]:
cancun_cube.mci.isel(time=0).plot(vmin=0, vmax=0.05)

In [ ]:
#the same as the first plot, but with dots
fig=plt.figure()
ax=fig.add_subplot(111)
ax.set_ylim(ymin=0., ymax=0.050)
cancun_cube.mci.sel(lat=21., lon=18., method="nearest").plot(ax=ax, marker ='o', linestyle = '', antialiased=True)
plt.savefig('mci_macro_cancun_2020.jpg')

In [ ]:
n=0

while n < 5:
    m=cancun_cube.mci.isel(time=n)
    m.plot(vmin=0, vmax=0.05, cmap= 'viridis', figsize = (10, 20))
    plt.savefig(str(n)+'chl_macro_cancun20.jpg')
    n=n+1